<a href="https://colab.research.google.com/github/enzo-augusto/Regressao-Simbolica-Paralela/blob/main/Regress%C3%A3o_Simb%C3%B3lica_Paralela.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Esse é o código do método 1 apresentado na monografia. Dois arquivos de texto são gerados ao rodar esse código. Um com os pontos do gráfico gerado pelo polinômio encontrado (graph3.txt) e outro com as informações relevantes sobre o resultado (**resultado3**.txt).


In [ ]:
%%writefile Ep4v1.cpp

#define _USE_MATH_DEFINES
#include <stdio.h>
#include <stdlib.h>
#include <time.h>
#include <unistd.h>
#include <math.h>
#include "mpi.h"
#include <stdbool.h>
#define gerMax 10000                     /**Interessante variar */
#define grauMax 20                       /**Interessante variar */
#define coefiMax 10                      /**Interessante variar */
#define pontosDominio 40                 /**Interessante variar */
#define tamanhoGeracao 100
#define inicio_dominio 0                 /**Interessante variar */
#define final_dominio 2*M_PI             /**Interessante variar */

#define ajusta_dominio (double)(final_dominio - inicio_dominio)/(double)pontosDominio

//Gera floats aleatorios dentro de um intervalo dado
double gera_random_float (double inicio_intervalo, double final_intervalo)
{
    return ((double) rand()/(double) RAND_MAX)* (final_intervalo - inicio_intervalo) + inicio_intervalo;
}

//Gera ints aleatorios dentro de um intervalo dado
int gera_random_int (int inicio_intervalo, int final_intervalo)
{
    return ( rand()%(final_intervalo - inicio_intervalo + 1) +  inicio_intervalo);
}

//Da o que precisamos para definir um polinomio. A variavel diferenca é a diferenca
//entre o polinomio e a funcao dada, e sera o que usaremos para ordenar os polinomios de uma geracao
typedef struct{
    int grau;
    double coeficientes[grauMax + 1];
    double diferenca;
    int rank;
}polinomio;

//inicializa o polinomio com valores aleatorios e zera a diferenca
void inicializa_polinomio(polinomio* poli)
{
    (*poli).grau = gera_random_int(0, grauMax);

    for(int i = 0; i <= grauMax; i++)
    {
        (*poli).coeficientes[i] = 0;
    }

    for(int i = 0; i <= (*poli).grau; i++)
    {
        (*poli).coeficientes[i] = gera_random_float(-coefiMax, coefiMax);
    }

    (*poli).diferenca = 0;
}

//Devolve um ponteiro para um vetor estatico com a imagem do polinomio
static double* gera_grafico(polinomio* poli)
{
    static double imagem[pontosDominio];
    int grau = (*poli).grau;
    for(int i = 0; i < pontosDominio; i++)
    {
        imagem[i]  = 0;
    }

    for(int i = 0; i < pontosDominio; i++)
    {
        for(int j = 0; j <= grau; j++)
        {
            imagem[i] = imagem[i] + (*poli).coeficientes[j]*pow((i*ajusta_dominio),(double)j);
        }
    }

    return imagem;
}

//retorna a soma das distancias entre as imagens da funcao dada e do polinomio
double calcula_diferenca(double funcao_dada[], double imagem_poli[])
{
    double diferenca = 0;

    for(int i = 0; i < pontosDominio; i++)
    {
        diferenca = diferenca + fabs(funcao_dada[i] - imagem_poli[i]);
    }
    return diferenca;
}

//retorna um polinomio mutado.
polinomio mutacao(polinomio poli)
{
    //se a diferenca é pequena, os coeficientes dos polinomios sao alterados a partir de uma distribuicao normal.
    //O grau pode ser aumentado ou coeficientes podem zer zerados também
    if(poli.diferenca < 3)                                                    //Interessante variar
    {
        for(int i = 0; i <= poli.grau; i++)
        {
            double novo_coefic = 0;

            for(int j = 0; j < 10; j++)
            {
            novo_coefic = novo_coefic + gera_random_float(poli.coeficientes[i] - 0.1, poli.coeficientes[i] + 0.1);
            }

            poli.coeficientes[i] = novo_coefic/10;
        }

        if(poli.grau < grauMax && gera_random_int(1, 10) >= 8)
        {
            poli.coeficientes[poli.grau + 1] = 1;
            poli.grau++;
        }

        if(poli.grau > 0 && gera_random_int(1, 10) >= 5)
        {
        int coeficiente_zerado = gera_random_int(0, poli.grau);
        poli.coeficientes[coeficiente_zerado] = 0;
        }

    }
    else if(poli.diferenca < 5)                                                    //Interessante variar
    {
        for(int i = 0; i <= poli.grau; i++)
        {
            double novo_coefic = 0;

            for(int j = 0; j < 10; j++)
            {
            novo_coefic = novo_coefic + gera_random_float(poli.coeficientes[i] - 0.2, poli.coeficientes[i] + 0.2);
            }

            poli.coeficientes[i] = novo_coefic/10;
        }

        if(poli.grau < grauMax && gera_random_int(1, 10) >= 8)
        {
            poli.coeficientes[poli.grau + 1] = 1;
            poli.grau++;
        }

        if(poli.grau > 0 && gera_random_int(1, 10) >= 5)
        {
        int coeficiente_zerado = gera_random_int(0, poli.grau);
        poli.coeficientes[coeficiente_zerado] = 0;
        }

    }
    else if(poli.diferenca < 10)                                                    //Interessante variar
    {
        for(int i = 0; i <= poli.grau; i++)
        {
            double novo_coefic = 0;

            for(int j = 0; j < 10; j++)
            {
            novo_coefic = novo_coefic + gera_random_float(poli.coeficientes[i] - 0.5, poli.coeficientes[i] + 0.5);
            }

            poli.coeficientes[i] = novo_coefic/10;
        }

        if(poli.grau < grauMax && gera_random_int(1, 10) >= 8)
        {
            poli.coeficientes[poli.grau + 1] = 1;
            poli.grau++;
        }

        if(poli.grau > 0 && gera_random_int(1, 10) >= 5)
        {
        int coeficiente_zerado = gera_random_int(0, poli.grau);
        poli.coeficientes[coeficiente_zerado] = 0;
        }

    }
    else if(poli.diferenca < 100)                                                    //Interessante variar
    {
        for(int i = 0; i <= poli.grau; i++)
        {
            double novo_coefic = 0;

            for(int j = 0; j < 10; j++)
            {
            novo_coefic = novo_coefic + gera_random_float(poli.coeficientes[i] - 1, poli.coeficientes[i] + 1);
            }

            poli.coeficientes[i] = novo_coefic/10;
        }

        if(poli.grau < grauMax && gera_random_int(1, 10) >= 8)
        {
            poli.coeficientes[poli.grau + 1] = 1;
            poli.grau++;
        }

        if(poli.grau > 0 && gera_random_int(1, 10) >= 5)
        {
        int coeficiente_zerado = gera_random_int(0, poli.grau);
        poli.coeficientes[coeficiente_zerado] = 0;
        }

    }
    else if( poli.diferenca < 1000)
    {
         for(int i = 0; i <= poli.grau; i++)
        {
            double novo_coefic = 0;

            for(int j = 0; j < 10; j++)
            {
            novo_coefic = novo_coefic + gera_random_float(poli.coeficientes[i] - 2, poli.coeficientes[i] + 2);
            }

            poli.coeficientes[i] = novo_coefic/10;
        }

        if(poli.grau < grauMax && gera_random_int(1, 10) >= 8)
        {
            poli.coeficientes[poli.grau + 1] = 1;
            poli.grau++;
        }

        if(poli.grau > 0 && gera_random_int(1, 10) >= 5)
        {
        int coeficiente_zerado = gera_random_int(0, poli.grau);
        poli.coeficientes[coeficiente_zerado] = 0;
        }

    }else{  //Se a diferenca e muito grande o polinomio e descartado e outro aleatorio toma seu lugar
        inicializa_polinomio(&poli);
    }

    return poli;
}

//Ordena os polinomios dentro de um vetor a partir de suas diferencas utilizando selection sort
void ordena_geracao(polinomio poli[])
{
    polinomio poli_aux[tamanhoGeracao];
    poli_aux[0] = poli[0];
    for(int i = 1; i < tamanhoGeracao; i++){
        for(int contador = i; contador >= 1; contador--){
            if (poli_aux[contador - 1].diferenca > poli[i].diferenca){
                poli_aux[contador] = poli_aux[contador - 1];
                poli_aux[contador - 1] = poli[i];
            }
            else{
                poli_aux[contador] = poli[i];
                break;
            }
        }
    }
    for(int i = 0; i < tamanhoGeracao; i++)
    {
        poli[i] = poli_aux[i];
    }
}

//Recebe o vetor de polinomios ja ordenado. Cria uma nova geracao a partir de mutacoes dos nove melhores resultados
void nova_geracao(polinomio poli[])
{
    for(int i = 0; i < 9; i++ )
    {
        for(int j = 0; j < 10; j++ )
        {
            poli[(i + 1)*10 + j] = mutacao(poli[i]);
        }

    }

}

int main(int argc, char **argv)
{

    //inicializando as variáveis específicas da versão MPI
    int MyRank, Numprocs;

    // Inicializando o ambiente de programação em paralelo
    MPI_Init(&argc, &argv);
    MPI_Comm_size(MPI_COMM_WORLD, &Numprocs);
    MPI_Comm_rank(MPI_COMM_WORLD, &MyRank);

    srand(time(NULL)*getpid() + MyRank);

    int geracao = 1;

    polinomio polinomios_vivos [tamanhoGeracao];

    polinomio todosPolinomios[Numprocs*tamanhoGeracao];

    // inicializa a funcao que iremos aproximar
    double funcao[pontosDominio];
    for(int i = 1; i < pontosDominio; i++)
    {
        funcao[i] = cos((i*ajusta_dominio)*(i*ajusta_dominio)) ;     /**Interessante variar. i deve sempre ser multiplicado pelo ajusta_dominio para agir como x */
    }

    //inicializa os polinomios do nosso vetor
    for(int i = 0; i < tamanhoGeracao; i++)
    {
        inicializa_polinomio(&polinomios_vivos[i]);
    }

    //gera os graficos dos polinomios e os compara com o da funcao que iremos aproximar
    for ( int i = 0; i < tamanhoGeracao; i++)
    {
        double* imagem_poli = gera_grafico(&polinomios_vivos[i]);
        polinomios_vivos[i].diferenca = calcula_diferenca(funcao, imagem_poli);
    }

    ordena_geracao(polinomios_vivos);

    //A parte paralela começa dentro deste loop. Cada processo é responsável por gerar uma nova geração a partir da antiga, eles então enviam todos os indivíduos
    //dessa geração para o rank 0, que os ordena e devolve os melhores para o resto dos processos. Os processos então criam uma nova geração a partir dos polinômios
    //recebidos e o loop se inicia novamente.
    while( geracao < gerMax)
    {
        nova_geracao(polinomios_vivos);

        for ( int i = 0; i < tamanhoGeracao; i++)
        {
            double* imagem_poli = gera_grafico(&polinomios_vivos[i]);
            polinomios_vivos[i].diferenca = calcula_diferenca(funcao, imagem_poli);
        }

        ordena_geracao(polinomios_vivos);

        if(MyRank != 0)
        {
            MPI_Send((void*)&polinomios_vivos, sizeof(polinomio)*tamanhoGeracao, MPI_CHAR, 0, 0, MPI_COMM_WORLD);
            MPI_Recv((void*)&polinomios_vivos, sizeof(polinomio)*tamanhoGeracao, MPI_CHAR, 0, 0, MPI_COMM_WORLD, MPI_STATUS_IGNORE);
        }
        else
        {
            for ( int i = 0; i < tamanhoGeracao; i++)
            {
                todosPolinomios[i] = polinomios_vivos[i];
            }

            for(int i = 1; i < Numprocs; i++)
            {
                MPI_Recv((void*)&todosPolinomios + i*sizeof(polinomio)*tamanhoGeracao, sizeof(polinomio)*tamanhoGeracao, MPI_CHAR, i, 0, MPI_COMM_WORLD, MPI_STATUS_IGNORE );
            }

            polinomio poli_aux[Numprocs*tamanhoGeracao];
            poli_aux[0] = todosPolinomios[0];

            for(int i = 1; i < Numprocs*tamanhoGeracao; i++)
            {
                for(int contador = i; contador >= 1; contador--)
                {
                    if (poli_aux[contador - 1].diferenca > todosPolinomios[i].diferenca)
                    {
                        poli_aux[contador] = poli_aux[contador - 1];
                        poli_aux[contador - 1] = todosPolinomios[i];
                    }
                    else
                    {
                        poli_aux[contador] = todosPolinomios[i];
                        break;
                    }
                }
            }

            for(int i = 0; i < Numprocs*tamanhoGeracao; i++)
            {
                todosPolinomios[i] = poli_aux[i];
            }

            for(int i = 0; i < 10; i++)
            {
                polinomios_vivos[i] = todosPolinomios[i];
            }

            for(int i = 1; i < Numprocs; i++)
            {
                MPI_Send((void*)&todosPolinomios + i*sizeof(polinomio)*10, sizeof(polinomio)*10, MPI_CHAR, i, 0, MPI_COMM_WORLD);
            }

        }

        geracao++;
    }


// O rank 0 salva todos os resultados
    if(MyRank == 0)
    {
    FILE* resultado3 = fopen("resultado3.txt", "w");
    if(resultado3 == NULL)
    {
        printf("Erro ao abrir arquivo de escrita.\n");
        return 1;
    }

    fprintf(resultado3, "Geração: %d \n", geracao);
    fprintf(resultado3, "Grau: %d \n", todosPolinomios[0].grau);

    for(int i = 0; i <= todosPolinomios[0].grau; i++)
    {
        fprintf(resultado3, "Coeficiente %d, %lf \n", i, todosPolinomios[0].coeficientes[i]);
    }

    fprintf(resultado3, "Diferença mínima %lf \n", todosPolinomios[0].diferenca);
    fprintf(resultado3, "Pontos no domínio %d \n", pontosDominio);

    FILE* graph3 = fopen("graph3.txt", "w");
    if(graph3 == NULL)
    {
        printf("Erro ao abrir arquivo de escrita.\n");
        return 1;
    }

    double* imagem = gera_grafico(&todosPolinomios[0]);

    for(int i = 0; i < pontosDominio; i++)
    {
        double dominio_certo = i*ajusta_dominio;
        fprintf(graph3, "%lf    %lf \n", dominio_certo, *(imagem + i));
    }

    fprintf(resultado3, "Tempo em segundos %lf \n", (double)clock()/CLOCKS_PER_SEC);
    fprintf(resultado3, "Numero de processadores %d \n", Numprocs);
    fclose(resultado3);
    fclose(graph3);
    }

    //Rank zero printa o tempo que o código demorou para rodar
    printf("%d: Tempo %lf \n", MyRank, (double)clock()/CLOCKS_PER_SEC);


    MPI_Finalize();

    printf("%d: deu bom", MyRank);
    return 0;
}



Writing Ep4v1.cpp


In [ ]:
!mpic++ Ep4v1.cpp -o Ep4v1
!mpiexec -n 2 --allow-run-as-root --oversubscribe ./Ep4v1

Ep4v1.cpp: In function ‘int main(int, char**)’:
Ep4v1.cpp:345:50: warning: pointer of type ‘void *’ used in arithmetic []8;;https://gcc.gnu.org/onlinedocs/gcc/Warning-Options.html#index-Wpointer-arith-Wpointer-arith]8;;]
  345 |                 MPI_Recv((void*)&todosPolinomios + i*sizeof(polinomio)*tamanhoGeracao, sizeof(polinomio)*tamanhoGeracao, MPI_CHAR, i, 0, MPI_COMM_WORLD, MPI_STATUS_IGNORE );
      |                                                  ^
Ep4v1.cpp:380:50: warning: pointer of type ‘void *’ used in arithmetic []8;;https://gcc.gnu.org/onlinedocs/gcc/Warning-Options.html#index-Wpointer-arith-Wpointer-arith]8;;]
  380 |                 MPI_Send((void*)&todosPolinomios + i*sizeof(polinomio)*10, sizeof(polinomio)*10, MPI_CHAR, i, 0, MPI_COMM_WORLD);
      |                          ~~~~~~~~~~~~~~~~~~~~~~~~^~~~~~~~~~~~~~~~~~~~~~~~
1: Tempo 10.688189 
0: Tempo 10.285462 
0: deu bom1: deu bom

Esse é o código do método 2 apresentado na monografia. Dois arquivos de texto são gerados ao rodar esse código. Um com os pontos do gráfico gerado pelo polinômio encontrado (graph2.txt) e outro com as informações relevantes sobre o resultado (**resultado2**.txt).

In [ ]:
%%writefile Ep4v2.cpp

#define _USE_MATH_DEFINES
#include <stdio.h>
#include <stdlib.h>
#include <time.h>
#include <unistd.h>
#include <math.h>
#include "mpi.h"
#include <stdbool.h>
#define gerMax 10000                     /**Interessante variar */
#define grauMax 20                       /**Interessante variar */
#define coefiMax 10                      /**Interessante variar */
#define pontosDominio 40                 /**Interessante variar */
#define tamanhoGeracao 100
#define inicio_dominio 0                 /**Interessante variar */
#define final_dominio 2*M_PI             /**Interessante variar */

#define ajusta_dominio (double)(final_dominio - inicio_dominio)/(double)pontosDominio

//Gera floats aleatorios dentro de um intervalo dado
double gera_random_float (double inicio_intervalo, double final_intervalo)
{
    return ((double) rand()/(double) RAND_MAX)* (final_intervalo - inicio_intervalo) + inicio_intervalo;
}

//Gera ints aleatorios dentro de um intervalo dado
int gera_random_int (int inicio_intervalo, int final_intervalo)
{
    return ( rand()%(final_intervalo - inicio_intervalo + 1) +  inicio_intervalo);
}

//Da o que precisamos para definir um polinomio. A variavel diferenca é a diferenca
//entre o polinomio e a funcao dada, e sera o que usaremos para ordenar os polinomios de uma geracao
typedef struct{
    int grau;
    double coeficientes[grauMax + 1];
    double diferenca;
    int rank;
}polinomio;

//inicializa o polinomio com valores aleatorios e zera a diferenca
void inicializa_polinomio(polinomio* poli)
{
    (*poli).grau = gera_random_int(0, grauMax);

    for(int i = 0; i <= grauMax; i++)
    {
        (*poli).coeficientes[i] = 0;
    }

    for(int i = 0; i <= (*poli).grau; i++)
    {
        (*poli).coeficientes[i] = gera_random_float(-coefiMax, coefiMax);
    }

    (*poli).diferenca = 0;
}

//Devolve um ponteiro para um vetor estatico com a imagem do polinomio
static double* gera_grafico(polinomio* poli)
{
    static double imagem[pontosDominio];
    int grau = (*poli).grau;
    for(int i = 0; i < pontosDominio; i++)
    {
        imagem[i]  = 0;
    }

    for(int i = 0; i < pontosDominio; i++)
    {
        for(int j = 0; j <= grau; j++)
        {
            imagem[i] = imagem[i] + (*poli).coeficientes[j]*pow((i*ajusta_dominio),(double)j);
        }
    }

    return imagem;
}

//retorna a soma das distancias entre as imagens da funcao dada e do polinomio
double calcula_diferenca(double funcao_dada[], double imagem_poli[])
{
    double diferenca = 0;

    for(int i = 0; i < pontosDominio; i++)
    {
        diferenca = diferenca + fabs(funcao_dada[i] - imagem_poli[i]);
    }
    return diferenca;
}

//retorna um polinomio mutado.
polinomio mutacao(polinomio poli)
{
    //se a diferenca é pequena, os coeficientes dos polinomios sao alterados a partir de uma distribuicao normal.
    //O grau pode ser aumentado ou coeficientes podem zer zerados também
    if(poli.diferenca < 3)                                                    //Interessante variar
    {
        for(int i = 0; i <= poli.grau; i++)
        {
            double novo_coefic = 0;

            for(int j = 0; j < 10; j++)
            {
            novo_coefic = novo_coefic + gera_random_float(poli.coeficientes[i] - 0.1, poli.coeficientes[i] + 0.1);
            }

            poli.coeficientes[i] = novo_coefic/10;
        }

        if(poli.grau < grauMax && gera_random_int(1, 10) >= 8)
        {
            poli.coeficientes[poli.grau + 1] = 1;
            poli.grau++;
        }

        if(poli.grau > 0 && gera_random_int(1, 10) >= 5)
        {
        int coeficiente_zerado = gera_random_int(0, poli.grau);
        poli.coeficientes[coeficiente_zerado] = 0;
        }

    }
    else if(poli.diferenca < 5)                                                    //Interessante variar
    {
        for(int i = 0; i <= poli.grau; i++)
        {
            double novo_coefic = 0;

            for(int j = 0; j < 10; j++)
            {
            novo_coefic = novo_coefic + gera_random_float(poli.coeficientes[i] - 0.2, poli.coeficientes[i] + 0.2);
            }

            poli.coeficientes[i] = novo_coefic/10;
        }

        if(poli.grau < grauMax && gera_random_int(1, 10) >= 8)
        {
            poli.coeficientes[poli.grau + 1] = 1;
            poli.grau++;
        }

        if(poli.grau > 0 && gera_random_int(1, 10) >= 5)
        {
        int coeficiente_zerado = gera_random_int(0, poli.grau);
        poli.coeficientes[coeficiente_zerado] = 0;
        }

    }
    else if(poli.diferenca < 10)                                                    //Interessante variar
    {
        for(int i = 0; i <= poli.grau; i++)
        {
            double novo_coefic = 0;

            for(int j = 0; j < 10; j++)
            {
            novo_coefic = novo_coefic + gera_random_float(poli.coeficientes[i] - 0.5, poli.coeficientes[i] + 0.5);
            }

            poli.coeficientes[i] = novo_coefic/10;
        }

        if(poli.grau < grauMax && gera_random_int(1, 10) >= 8)
        {
            poli.coeficientes[poli.grau + 1] = 1;
            poli.grau++;
        }

        if(poli.grau > 0 && gera_random_int(1, 10) >= 5)
        {
        int coeficiente_zerado = gera_random_int(0, poli.grau);
        poli.coeficientes[coeficiente_zerado] = 0;
        }

    }
    else if(poli.diferenca < 100)                                                    //Interessante variar
    {
        for(int i = 0; i <= poli.grau; i++)
        {
            double novo_coefic = 0;

            for(int j = 0; j < 10; j++)
            {
            novo_coefic = novo_coefic + gera_random_float(poli.coeficientes[i] - 1, poli.coeficientes[i] + 1);
            }

            poli.coeficientes[i] = novo_coefic/10;
        }

        if(poli.grau < grauMax && gera_random_int(1, 10) >= 8)
        {
            poli.coeficientes[poli.grau + 1] = 1;
            poli.grau++;
        }

        if(poli.grau > 0 && gera_random_int(1, 10) >= 5)
        {
        int coeficiente_zerado = gera_random_int(0, poli.grau);
        poli.coeficientes[coeficiente_zerado] = 0;
        }

    }
    else if( poli.diferenca < 1000)
    {
         for(int i = 0; i <= poli.grau; i++)
        {
            double novo_coefic = 0;

            for(int j = 0; j < 10; j++)
            {
            novo_coefic = novo_coefic + gera_random_float(poli.coeficientes[i] - 2, poli.coeficientes[i] + 2);
            }

            poli.coeficientes[i] = novo_coefic/10;
        }

        if(poli.grau < grauMax && gera_random_int(1, 10) >= 8)
        {
            poli.coeficientes[poli.grau + 1] = 1;
            poli.grau++;
        }

        if(poli.grau > 0 && gera_random_int(1, 10) >= 5)
        {
        int coeficiente_zerado = gera_random_int(0, poli.grau);
        poli.coeficientes[coeficiente_zerado] = 0;
        }

    }else{  //Se a diferenca e muito grande o polinomio e descartado e outro aleatorio toma seu lugar
        inicializa_polinomio(&poli);
    }

    return poli;
}

//Ordena os polinomios dentro de um vetor a partir de suas diferencas utilizando selection sort
void ordena_geracao(polinomio poli[])
{
    polinomio poli_aux[tamanhoGeracao];
    poli_aux[0] = poli[0];
    for(int i = 1; i < tamanhoGeracao; i++){
        for(int contador = i; contador >= 1; contador--){
            if (poli_aux[contador - 1].diferenca > poli[i].diferenca){
                poli_aux[contador] = poli_aux[contador - 1];
                poli_aux[contador - 1] = poli[i];
            }
            else{
                poli_aux[contador] = poli[i];
                break;
            }
        }
    }
    for(int i = 0; i < tamanhoGeracao; i++)
    {
        poli[i] = poli_aux[i];
    }
}

//Recebe o vetor de polinomios ja ordenado. Cria uma nova geracao a partir de mutacoes dos nove melhores resultados
void nova_geracao(polinomio poli[])
{
    for(int i = 0; i < 9; i++ )
    {
        for(int j = 0; j < 10; j++ )
        {
            poli[(i + 1)*10 + j] = mutacao(poli[i]);
        }
    }

}

int main(int argc, char **argv)
{

    //inicializando as variáveis específicas da versão MPI
    int MyRank, Numprocs, tag;

    // Inicializando o ambiente de programação em paralelo
    MPI_Init(&argc, &argv);
    MPI_Comm_size(MPI_COMM_WORLD, &Numprocs);
    MPI_Comm_rank(MPI_COMM_WORLD, &MyRank);

    srand(time(NULL)*getpid() + MyRank);

    int geracao = 1;

    float dif_min;

    float dif_esp = 0.1; /**Interessante variar */

    polinomio polinomios_vivos [tamanhoGeracao];
    polinomio melhoresPolinomios[Numprocs];

    // inicializa a funcao que iremos aproximar
    double funcao[pontosDominio];
    for(int i = 1; i < pontosDominio; i++)
    {
        funcao[i] = exp((i*ajusta_dominio));     /**Interessante variar. i deve sempre ser multiplicado pelo ajusta_dominio para agir como x */
    }

    //inicializa os polinomios do nosso vetor
    for(int i = 0; i < tamanhoGeracao; i++)
    {
        inicializa_polinomio(&polinomios_vivos[i]);
    }

    //gera os graficos dos polinomios e os compara com o da funcao que iremos aproximar
    for ( int i = 0; i < tamanhoGeracao; i++)
    {
        double* imagem_poli = gera_grafico(&polinomios_vivos[i]);
        polinomios_vivos[i].diferenca = calcula_diferenca(funcao, imagem_poli);
    }

    ordena_geracao(polinomios_vivos);
    dif_min = polinomios_vivos[0].diferenca;

    while(dif_min > dif_esp && geracao < gerMax)
    {
        nova_geracao(polinomios_vivos);

        for ( int i = 0; i < tamanhoGeracao; i++)
        {
            double* imagem_poli = gera_grafico(&polinomios_vivos[i]);
            polinomios_vivos[i].diferenca = calcula_diferenca(funcao, imagem_poli);
        }

        ordena_geracao(polinomios_vivos);
        dif_min = polinomios_vivos[0].diferenca;
        geracao++;

    }

    //Aqui começa a mudança da parte paralela. Enviamos os polinômios de cada processo para o processo 0, ele então acha o melhor entre todos. Só isso
    polinomios_vivos[0].rank = MyRank;

    if(MyRank != 0)
    {
        MPI_Send((void*)&polinomios_vivos[0], sizeof(polinomio), MPI_CHAR, 0, 0, MPI_COMM_WORLD);
    }
    else
    {
        melhoresPolinomios[0] = polinomios_vivos[0];

        for(int i = 1; i < Numprocs; i++)
        {
            MPI_Recv((void*)&melhoresPolinomios[i], sizeof(polinomio), MPI_CHAR, i, 0, MPI_COMM_WORLD, MPI_STATUS_IGNORE);
        }
        polinomio poli_aux[Numprocs];
        poli_aux[0] = melhoresPolinomios[0];
        for(int i = 1; i < Numprocs; i++)
        {
            for(int contador = i; contador >= 1; contador--)
            {
                if (poli_aux[contador - 1].diferenca > melhoresPolinomios[i].diferenca)
                {
                    poli_aux[contador] = poli_aux[contador - 1];
                    poli_aux[contador - 1] = melhoresPolinomios[i];
                }
                else
                {
                    poli_aux[contador] = melhoresPolinomios[i];
                    break;
                }
        }
    }
    for(int i = 0; i < Numprocs; i++)
    {
        melhoresPolinomios[i] = poli_aux[i];
    }

    FILE* resultado2 = fopen("resultado2.txt", "w");
    if(resultado2 == NULL)
    {
        printf("Erro ao abrir arquivo de escrita.\n");
        return 1;
    }

    fprintf(resultado2, "Geração: %d \n", geracao);
    fprintf(resultado2, "Grau: %d \n", melhoresPolinomios[0].grau);

    for(int i = 0; i <= melhoresPolinomios[0].grau; i++)
    {
        fprintf(resultado2, "Coeficiente %d, %lf \n", i, melhoresPolinomios[0].coeficientes[i]);
    }

    fprintf(resultado2, "Diferença mínima %lf \n", melhoresPolinomios[0].diferenca);
    fprintf(resultado2, "Pontos no domínio %d \n", pontosDominio);
    fprintf(resultado2, "Tempo em segundos %lf \n", (double)clock()/CLOCKS_PER_SEC);
    fprintf(resultado2, "Numero de processadores %d \n", Numprocs);

    FILE* graph2 = fopen("graph2.txt", "w");
    if(graph2 == NULL)
    {
        printf("Erro ao abrir arquivo de escrita.\n");
        return 1;
    }

    double* imagem = gera_grafico(&melhoresPolinomios[0]);

    for(int i = 0; i < pontosDominio; i++)
    {
        double dominio_certo = i*ajusta_dominio;
        fprintf(graph2, "%lf    %lf \n", dominio_certo, *(imagem + i));
    }

    fclose(resultado2);
    fclose(graph2);
    }



    MPI_Finalize();

    printf("%d: deu bom", MyRank);
    return 0;
}



Overwriting Ep4v2.cpp


In [ ]:
!mpic++ Ep4v2.cpp -o Ep4v2
!mpiexec -n 2 --allow-run-as-root --oversubscribe ./Ep4v2

1: deu bom0: deu bom